In [1]:
import cStringIO
import string
import adabas
from adabas.api import *
from adabas.datamap import *

In [2]:
from datetime import date
mask_date = lambda data, mask='%d-%m-%Y': date.fromordinal(int(data[2:])-364).strftime(mask)

In [3]:
DBID=12;FNR=11 # EmplTel file
STARTISN=201
RCOUNT=100
# define the mapping of data in record buffer to attributes
# of EmpTel class

# create datamap object for Employees-Telephone-List
emp=Datamap('EmplTel',
    String('personnel_id',  8),
    String('firstname',    20),
    String('m_initial',    20),
    String('lastname',     20),
    String('birth',         8),
    String('country',       3),
    String('areacode',      6),
    String('phone',        15),
    String('department',    6),
    String('jobtitle',     25),
    String('ai1',          20),
    String('ai2',          20),
    String('ai3',          20),        
    String('az1',           3),        
    String('az2',           3))

In [4]:
lines=''                                         
extraline=''                                     
alter=False                                      
c1=Adabas(rbl=256,fbl=64,sbl=32,vbl=128,ibl=0)   
c1.cb.dbid=DBID
c1.cb.fnr=FNR
c1.cb.cid='1010'                                 
c1.fb.value='AA,AC,AD,AE,AH,8,U,AL,AN,AM,AO,AP,AI1-3,AZ1-2.' 


In [5]:
c1.cb.isn=STARTISN                                                                 
# use emp Datamap on record buffer                                                 
emp.buffer=c1.rb                                                                   
emp.offset=0                                                                       
                                                                                  
count=0                                                                            
                                                                                   
try:                                                                               
    for count in range(RCOUNT):                                                    
        c1.readByIsn(getnext=1)                                                    
                                                                                    
        lines += "{} - {:35} - {} -ai1 {} -ai2 {} -ai3 {} - {} - {}\n".format(
                emp.personnel_id,
                string.capwords(emp.lastname+', '+emp.firstname+' '+emp.m_initial),
                mask_date(emp.birth, mask ='%d/%m/%Y'),
               # emp.country,
               # emp.areacode,
               # emp.phone,
               # emp.department,
               # string.capwords(emp.jobtitle),
                emp.ai1, emp.ai2, emp.ai3, emp.az1, emp.az2)
    lines+= 'Sequential Read by ISN returned '+str(count+1)+' record(s).'
except DataEnd:
    extraline+= 'Sequential Read by ISN returned '+str(count)+' record(s).'
    pass
except DatabaseError, (line, apa):
    extraline+='Database Error:'+line
                

In [6]:
print lines


11400327 - Speck, Frank Heinz                  - 22/07/1952 -ai1 SCHILLERSTR.5 -ai2 6100 DARMSTADT -ai3  - GER - 
11400328 - Schilling, Anna Hedwig              - 12/01/1949 -ai1 KASTANIENAL.12 -ai2 6100 DARMSTADT -ai3  - GER - 
11100102 - Schindler, Edgar Peter              - 04/12/1962 -ai1 BUCHENLANDWEG 84 -ai2 6148 HEPPENHEIM -ai3  - GER - ENG
11100105 - Schirm, Christian                   - 15/03/1961 -ai1 SIEMENSSTR.4 -ai2 6100 DARMSTADT -ai3  - GER - ENG
11100106 - Schmitt, Reiner Georg               - 13/02/1955 -ai1 WORMSERSTR.23 -ai2 6100 DARMSTADT -ai3  - GER - ENG
11100107 - Schmidt, Helga Gerda                - 26/05/1961 -ai1 AM ELFENGRUND 3 -ai2 6148 HEPPENHEIM -ai3  - GER - FRE
11100108 - Schneider, Wolfgang                 - 22/04/1957 -ai1 DORNHEIMERWEG 12 -ai2 6100 GRIESHEIM -ai3  - GER - 
11100109 - Schneider, Christa Eva-maria        - 23/11/1962 -ai1 DORNHEIMERWEG 23 -ai2 6100 GRIESHEIM -ai3  - GER - FRE
11100110 - Bungert, Georg Peter                - 16/03/1957 

In [ ]:
print extraline
